<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/MyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Reshape, Flatten
from keras.models import Model
import numpy as np

class MyModel():
  def __init__(self, data_path, max_input_len = 100, max_item_output_len = 50, max_output_len = 100):
    self.max_item_output_len = max_item_output_len
    self.max_input_len = max_input_len
    self.max_output_len = max_output_len

    self.data = self.get_data(data_path)
    self.model = self.get_model()

  def get_data(self,data_path):
    return {
    'input':[
        'dog playing',
        'cat playing',
        'lion playing'
        ],
    'output':[
        ['dog playing', 'dog x'],
        ['cat playing',' cat x'],
        ['lion playing' , 'lion x']
        ],
    'types':
     [
        [4,6],
        [4,6],
        [4,6]
        ]
      }

  def get_train(self):
    # Crea un objeto Tokenizer
    tokenizer = Tokenizer()
    # Ajusta el tokenizer a tus datos
    types =  self.data['types']
    output = self.data ['output']
    input =  self.data ['input']

    tokenizer.fit_on_texts(input+['x'])
    vocab_size = len(tokenizer.word_index) + 1
    sequences_input = tokenizer.texts_to_sequences(input)

    sequences_output = []
    for item in output:
      texts = [text for text in item]
      sequences_output.append(tokenizer.texts_to_sequences(texts))

    train_x =  [np.array([date_input, date_type]) for date_input, date_type in zip(sequences_input, types)]
    train_x = np.array(train_x)
    train_y = np.array( [np.array(item) for item in sequences_output])

    for item0 in train_x:
      for item1 in item0:
        for item2 in item1:
          while len(item2) < self.max_input_len:
            item1 = np.append(item1, -1)
    
    for item0 in train_y:
      for item1 in item0:
        while len(item1) < self.max_output_len:
          item1 = np.append(item1, np.array([[-1]* self.max_item_output_len]), axis=0)
        for item2 in item1:
          while len(item2) < self.max_item_output_len:
            item1 = np.append(item1, -1)
    
    return  {'train_x':train_x, 'train_y':train_y}

  def get_model(self):
    total_len_output = self.max_output_len * self.max_item_output_len
    inputA = Input(shape=(2, self.max_input_len))
    x = Flatten(input_shape=(2, self.max_input_len))(inputA)
    x = Dense(32, activation='relu')(x)
    x = Dense(4, activation='relu')(x)
    x = Dense(total_len_output, activation='relu')(x)
    output = Reshape((self.max_output_len, self.max_item_output_len))(x)
    return Model(inputs=inputA, outputs=output)

  def fit(self):
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    train = self.get_train()
    train_x = train['train_x']
    train_y = train['train_y']
    self.model.fit(train_x, train_y, epochs=10, batch_size=32)
    # Convierte tus datos en secuencias de números


[[[2 1]
  [4 6]]

 [[3 1]
  [4 6]]

 [[4 1]
  [4 6]]]
[[[2 1]
  [2 5]]

 [[3 1]
  [3 5]]

 [[4 1]
  [4 5]]]
